<a href="https://colab.research.google.com/github/jinkyukim-me/Pytorch_Class_/blob/master/10_2_mnist_cnn_deep_tutorial_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
import torch.nn as nn
import torch.nn.init
import torch.optim as optim
import torchvision.datasets as dsets
import torchvision.transforms as transforms

In [0]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [3]:
print(device)

cuda


In [4]:
torch.cuda.is_available()

True

In [0]:
#parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [0]:
#MNIST dataset
mnist_train = dsets.MNIST(root = 'MNIST_data/',
                         train = True,
                         transform = transforms.ToTensor(),
                         download = True)
mnist_test = dsets.MNIST(root = 'MNIST_data/',
                        train = False,
                        transform = transforms.ToTensor(),
                        download = True)

In [0]:
#Dataloader
data_loader = torch.utils.data.DataLoader(dataset = mnist_train,
                                         batch_size = batch_size,
                                         shuffle = True,
                                         drop_last = True)

In [0]:
#model
class CNN(nn.Module):
    
    def __init__(self):
        super(CNN, self).__init__()
        #Layer1
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size = 3 , padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        #Layer2
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size = 3, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        #Layer3
        self.layer3 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size = 3, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        #Layer4
        self.fc1 = nn.Linear(3*3*128, 625)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(625, 10, bias = True) #0~9:10
        torch.nn.init.xavier_uniform_(self.fc1.weight)
        torch.nn.init.xavier_uniform_(self.fc2.weight)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        
        out = out.view(out.size(0), -1)
        out = self.fc1(out)
        out = self.relu(out)
        out = self.fc2(out)
        return out
        

In [0]:
model = CNN().to(device)

In [10]:
value = torch.Tensor(1,1,28,28).to(device)
print((model(value).shape))

torch.Size([1, 10])


In [11]:
model

CNN(
  (layer1): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer3): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc1): Linear(in_features=1152, out_features=625, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=625, out_features=10, bias=True)
)

In [0]:
#loss function
criterion = nn.CrossEntropyLoss().to(device)

In [0]:
#optimizer
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [14]:
#training
total_batch = len(data_loader)
print('Learning Started. It will takes some time.')

for epoch in range(training_epochs):
    avg_cost = 0
    
    for X, Y in data_loader: #X:input Y:label
        X = X.to(device) #input data
        Y = Y.to(device) #label data
        
        optimizer.zero_grad()
        hypothesis = model(X) #model에 input값을 넣으면 가설이 됨
        
        cost = criterion(hypothesis, Y) #가설과 실제 라벨 사이의 차이를 계산하는 loss
        cost.backward()
        optimizer.step()
        
        avg_cost += cost / total_batch #cost값을 쌓아줌
        
    print('[Epoch:{}] cost = {}'.format(epoch +1, avg_cost))
print('Learning Finished!')

Learning Started. It will takes some time.
[Epoch:1] cost = 0.16196729242801666
[Epoch:2] cost = 0.04254630208015442
[Epoch:3] cost = 0.03063405118882656
[Epoch:4] cost = 0.023265168070793152
[Epoch:5] cost = 0.01881580613553524
[Epoch:6] cost = 0.015385657548904419
[Epoch:7] cost = 0.012677283026278019
[Epoch:8] cost = 0.011379619129002094
[Epoch:9] cost = 0.009857913479208946
[Epoch:10] cost = 0.009968802332878113
[Epoch:11] cost = 0.006669596303254366
[Epoch:12] cost = 0.007283030077815056
[Epoch:13] cost = 0.005461142864078283
[Epoch:14] cost = 0.006233238615095615
[Epoch:15] cost = 0.004821704234927893
Learning Finished!


In [15]:
#test
with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)
    
    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print("Accuracy:",accuracy.item())

Accuracy: 0.9878999590873718


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:58: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:48: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")
